# Problema





> Dada su alta letalidad (el COVID-19 registra aprox. un 7% comparada con otros virus), es importante determinar las condiciones de pacientes que pueden conllevar mayores factores de vulnerabilidad para asi poder tomar acciones de prevencion y cuidado dirigidas a ciertos sectores sociodemograficos de la poblacion en EEUU.


Hipotésis


>Crear un modelo de prediccion que nos permita determinar que factores sociodemograficos conllevan un mayor grado de vulnerabilidad midiendo la tasa de causalidad de eventos (necesidad de internamiento en hospital, ingreso a UCI, fallecimiento).
---
>Un modelo de forecasting, basandonos en los factores temporales (estacionalidad del tiempo,medidas preventivas del gobierno) y sociales (edad,sexo,raza) disponibles nos puede permitir realizar proyecciones en un futuro proximo (semanal,1M,3M), lo cual permite tomar acciones de planificacion por parte del sector sanitario.
---
>Encontrar atributos y patrones que inciden con mayor fuerza en la letalidad de la enfermedad (edad,sexo,raza,enfermedades preexistentes,ingreso a UCI)
para la predicción del riesgo de muerte en el paciente.

Análisis descriptivo - COVID 19

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [ ]:
data = pd.read_csv('../input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv')

#Descripción de los atributos del dataset
* **cdc_report_dt**= Fecha en que se informó el CDC (Center for Disease Control and Prevention)
* **pos_spec_dt**= Fecha de la primera recolección de muestras positivas
* **onset_dt**= ¿Cuál fue la fecha de inicio?
* **current_status**= ¿Cuál es el estado actual de esta persona?
* **sex**= género
* **age_group**= Categorías de grupos por edad
* **Race and ethnicity (combined)**= caso demográfico
* **hosp_yn**= ¿Fue hospitalizado el paciente?
* **icu_yn**= ¿El paciente fue ingresado en una unidad de cuidados intensivos (UCI)?
* **death_yn**= ¿Murió el paciente como consecuencia de esta enfermedad?
* **medcond_yn**=¿Tenían alguna condición médica subyacente y / o conductas de riesgo?

#Primero, recategorizamos correctamente las variables y damos una exploración preliminar de los datos.
Fechas : cdc_report_dt, pos_spec_dt, onset_dt <br>
Nominales : sex, current_status, Race and ethnicity <br>
Ordinales : age_group <br>
Targets : hosp_yn, icu_yn, death_yn, medcod_yn

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data['cdc_report_dt'] = pd.to_datetime(data['cdc_report_dt'])
data['pos_spec_dt'] = pd.to_datetime(data['pos_spec_dt'])
data['onset_dt'] = pd.to_datetime(data['onset_dt'])

In [ ]:
edad_dict = ['0 - 9 Years','10 - 19 Years','20 - 29 Years','30 - 39 Years','40 - 49 Years',
            '50 - 59 Years','60 - 69 Years','70 - 79 Years','80+ Years','Unknown']
data['age_group'] = pd.Categorical(data['age_group'],categories=edad_dict,ordered=True)

In [ ]:
data.dtypes

Desagrupamos la variable conjunta de etnia y raza para ver si existe info. que rescatar: solo vale la pena la raza, ya que etnia esta seteado a NA o 'Non-Hispanic', recategorizamos los  'Missing' a 'Unknown'.

In [ ]:
data['Race'], data['Ethnicity'] = data['Race and ethnicity (combined)'].str.split(', ',1).str
data.head()

In [ ]:
data['Race'].value_counts(normalize=True)

# Nueva sección

In [ ]:
data['Ethnicity'].value_counts(normalize=True)

In [ ]:
data['Race'] = ['Unknown' if x=='Missing' else x for x in data['Race']]
data = data.drop(['Race and ethnicity (combined)','Ethnicity'],axis=1)

In [ ]:
data.head()

In [ ]:
for i in data.columns[3:]:
  print('Nulos    '+ str(data[i].isnull().sum()))
  print(data[i].value_counts(normalize=True))
  print('-------------------------------')

In [ ]:
data['sex'] = ['Unknown' if x=='Missing' else x for x in data['sex']]

In [ ]:
print(data.shape)

In [ ]:
data.columns.values.tolist()

In [ ]:
data.describe()

Revisamos si es que hay valores faltantes (null)

In [ ]:
data.isnull().sum(axis=0)/data.shape[0]*100

Veamos la curva de casos reportados, positivos y fallecidos al mismo tiempo

In [ ]:
tabla_fechas_cdc = data['cdc_report_dt'].value_counts().sort_index(ascending=True).cumsum() # Curva acumulada de casos reportados
tabla_fechas_pos = data['pos_spec_dt'].value_counts().sort_index(ascending=True).cumsum() # Curva acumulada de positivos
tabla_fechas_muertos = data[data['death_yn']=='Yes']['cdc_report_dt'].value_counts().sort_index(ascending=True).cumsum() # Curva acumulada de casos fallecidos

In [ ]:
#tabla_fechas_cdc_no_acum = data['cdc_report_dt'].value_counts() 
#tabla_fechas_pos_no_acum = data['pos_spec_dt'].value_counts()
tabla_fechas_muertos_no_acum = data[data['death_yn']=='Yes']['cdc_report_dt'].value_counts()
fechas = np.unique(tabla_fechas_muertos_no_acum.head(3).index.tolist())
fechas_aux = [x.strftime("%Y-%m-%d") for x in fechas]
muertos = tabla_fechas_muertos_no_acum.filter(fechas).values.tolist()
muertos = [str(x/100)+' K' for x in muertos]
#fechas = np.unique(tabla_fechas_cdc_no_acum.head(5).index.tolist() + tabla_fechas_muertos_no_acum.head(5).index.tolist())

In [ ]:
def millions_formatter(x, pos):
    return f'{x/1000}'

In [ ]:
plt.rcParams['figure.figsize'] = (23,8)
plt.rcParams['xtick.labelsize']=14
plt.rcParams['ytick.labelsize']=14
plt.rc_context({'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'})
fig, ax = plt.subplots()
ax.grid()
lns1 = ax.plot(tabla_fechas_cdc.index,tabla_fechas_cdc,color='blue',linestyle='solid',markersize=7,label='Reportados',linewidth=3)
lns2 = ax.plot(tabla_fechas_pos.index,tabla_fechas_pos,color='red',linestyle='dotted',markersize=7,label='Positivos',linewidth=3)
ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))
ax2 = ax.twinx()
lns3 = ax2.plot(tabla_fechas_muertos.index,tabla_fechas_muertos,color='green',linestyle='dashed',markersize=7,label='Fallecidos',linewidth=3)
ax2.yaxis.set_major_formatter(FuncFormatter(millions_formatter))
ax.set_title('Curva acumulada de casos reportados \n Enero - Diciembre \n',fontsize=18)
ax.set_xlabel('\n Fecha',fontsize=16)
ax.set_ylabel('# Casos confirmados \n (en miles)',fontsize=16)
ax2.set_ylabel('\n # Casos fallecidos',fontsize=16)

for (i,j,k) in zip(fechas,muertos,fechas_aux):
  ax.axvline(x=i,color='orange',linestyle='dashdot',linewidth=2)
  ax.text(i,7600000,k+'\n'+j,rotation='vertical',ha='center',va='center',fontsize=13,backgroundcolor='white')

# Unica leyenda
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper left',fontsize=16)

Se puede observar un crecimiento empinado en los meses de abril, junio, setiembre y noviembre lo que coincide con las noticias reportadas en esos meses. (*https://cutt.ly/3jMgthg* : '**Coronavirus en Estados Unidos: 3 claves del "preocupante repunte de contagios**" ', *https://cutt.ly/ujMguiX* : '**Estados Unidos registra 968 muertes por coronavirus en un día**')

Adentrandonos en el mes que se registran los mayores picos de muertos.

In [ ]:
data_critica = data[(data.cdc_report_dt >= pd.to_datetime('2020-06-01')) & (data.cdc_report_dt<= pd.to_datetime('2020-06-30'))] # 880134 filas

Cuenta con 880,134 filas, cabe resaltar la cantidad de casos confirmados x laboratorio (840,466 -> aprox. 95%), vale la pena compara este mes con los demas meses. (Tomar en consideracion # de casos por confirmar en mayo)

In [ ]:
data_critica.describe()

In [ ]:
data['Periodo'] = pd.to_datetime(data['cdc_report_dt']).dt.to_period('M')

In [ ]:
data_agrupada = data.groupby(by=['Periodo','current_status']).size().unstack()
data_print = round(data_agrupada.div(data_agrupada.sum(axis=1),axis=0)*100,1)

In [ ]:
data_print

In [ ]:
data_agrupada = data[data['medcond_yn'].isin(['Yes','No'])].groupby(by=['Periodo','medcond_yn']).size().unstack()
data_print = round(data_agrupada.div(data_agrupada.sum(axis=1),axis=0)*100,1)
data_print = data_print.join(data_agrupada,rsuffix='_total')
data_print['Total'] = data_print['No_total'] + data_print['Yes_total']
data_print

Analizamos los atributos 

In [ ]:
grafica = sns.catplot(data=data[data.death_yn.isin(['Yes','No']) & data.sex.isin(['Male','Female']) ],
              x='age_group', 
              col='death_yn',
              hue='sex',
              kind='count',
              height=6, aspect=1.3)
grafica.set_xticklabels(rotation=30)

In [ ]:
grafica = sns.catplot(data=data[data.death_yn.isin(['Yes','No']) & data.sex.isin(['Male','Female']) ],
              y='Race', 
              col='death_yn',
              hue='sex',
              kind='count',
              order=data['Race'].value_counts().index,
              height=6, aspect=1.3)

Analizando posible variable a predecir.

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('medcond_yn - condicion medica subyacente ',fontsize = 20)
data['medcond_yn'].value_counts().plot.pie(autopct="%1.1f%%")

data['medcond_yn'].value_counts()

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('death_yn - Mortalidad del paciente',fontsize = 20)
data['death_yn'].value_counts().plot.pie(autopct="%1.1f%%")

data['death_yn'].value_counts()

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('hosp_yn - Hospitalización del paciente',fontsize = 20)
data['hosp_yn'].value_counts().plot.pie(autopct="%1.1f%%")

data['death_yn'].value_counts()

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('icu_yn - Paciente pasa a UCI',fontsize = 20)
data['icu_yn'].value_counts().plot.pie(autopct="%1.1f%%")

data['icu_yn'].value_counts()

Se observaron mucha cantidad de datos faltantes en las variables de fechas **pos_spec_dt** y	**onset_dt** por eso procedemos a eliminarlas


In [ ]:
data = data.drop(['pos_spec_dt','onset_dt'],axis =1)
print(data.shape)

También procedemos a elimnar los valores faltantes al ser un numero minimo de casos respecto a la base

In [ ]:
data.isnull().sum(axis=0)

In [ ]:
data = data.dropna()

data.isnull().sum(axis=0)

Se observa que ahora no hay datos faltantes y se tiene una base con 8404990 registros.

In [ ]:
data

In [ ]:
data.columns.tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
LB_Encode = LabelEncoder()

In [ ]:
data['current_status']=LB_Encode.fit_transform(data['current_status'])
data['sex']=LB_Encode.fit_transform(data['sex'])
data['age_group']=LB_Encode.fit_transform(data['age_group'])
data['Race']=LB_Encode.fit_transform(data['Race'])
data['hosp_yn']=LB_Encode.fit_transform(data['hosp_yn'])
data['icu_yn']=LB_Encode.fit_transform(data['icu_yn'])
data['death_yn']=LB_Encode.fit_transform(data['death_yn'])
data['medcond_yn']=LB_Encode.fit_transform(data['medcond_yn'])

In [ ]:
data.drop(['cdc_report_dt','Periodo'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data_ = data.copy()

In [ ]:
from sklearn import decomposition, datasets
from sklearn.preprocessing import StandardScaler

In [ ]:
data_.dropna(inplace=True)

In [ ]:
y = data_.pop('death_yn')
X = data_

In [ ]:
from sklearn.preprocessing import scale

#Scaling the values
X.iloc[:,:] = scale(X.iloc[:,:])

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
hpc = PCA(n_components=7).fit(X.iloc[:,:])
hpc

In [ ]:
hpc.explained_variance_ratio_

In [ ]:
var1=np.cumsum(np.round(hpc.explained_variance_ratio_, decimals=4)*100)
var1

In [ ]:
hpc = PCA(n_components=5).fit(X.iloc[:,:])
hpc

In [ ]:
var1=np.cumsum(np.round(hpc.explained_variance_ratio_, decimals=4)*100)
var1

In [ ]:
hpc = PCA(n_components=5).fit_transform(X)
hpc[0:5,]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(hpc,y,test_size=0.33, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
prediction=rf.predict(x_test)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

In [ ]:
data.columns.tolist()

In [ ]:
X = data[['current_status',
 'sex',
 'age_group',
 'hosp_yn',
 'icu_yn',
 'medcond_yn',
 'Race']]
y = data[['death_yn']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
prediction=rf.predict(x_test)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

**Antecedente**: Adquisición de conocimiento sobre la letalidad de la COVID-19 mediante técnicas de inteligencia artificial

 http://www.revistaccuba.sld.cu/index.php/revacc/article/view/891/912